In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from scombi_do.app import main
import gdal
import os
from urllib.parse import urlparse
import requests
from requests.auth import HTTPBasicAuth
from pystac import STAC_IO, read_file, Catalog, CatalogType, Item, Catalog
from ipyleaflet import Map, ImageOverlay
import numpy as np
from PIL import Image
from io import BytesIO
from base64 import b64encode
from shapely.geometry import shape

### Stage the STAC catalogs

In [3]:
def my_read_method(uri):
    
    parsed = urlparse(uri)
    
    if parsed.scheme.startswith('http'):
    
        if os.environ.get('STAGEIN_PASSWORD') is None:
            
            return requests.get(uri).text
            
        else:
            
            return requests.get(uri, 
                                auth=HTTPBasicAuth(os.environ.get('STAGEIN_USERNAME'), 
                                                   os.environ.get('STAGEIN_PASSWORD'))
                               ).text
    else:
        return STAC_IO.default_read_text_method(uri)


In [4]:
input_references = ['https://terradue-rtd.gitlab.io/sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414.json', 
                    'https://terradue-rtd.gitlab.io/sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348/S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348.json',
                    'https://terradue-rtd.gitlab.io/sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720.json',
                    'https://terradue-rtd.gitlab.io/sentinel-s2-l2a-cogs/48/M/YT/S2A_MSIL2A_20200906T025551_N0214_R032_T48MYT_20200906T063429/S2A_MSIL2A_20200906T025551_N0214_R032_T48MYT_20200906T063429.json']

In [5]:
STAC_IO.read_text_method = my_read_method
    
catalogs = []

for index, input_reference in enumerate(input_references):

    items = []
    
    thing = read_file(input_reference)

    if isinstance(thing, Item):

        items.append(thing)

    elif isinstance(thing, Catalog):

        for item in thing.get_items():

            items.append(item)

    # create catalog
    catalog = Catalog(id=items[0].id,
              description='staged STAC catalog with {}'.format(items[0].id))

    catalog.add_items(items)

    catalog.normalize_and_save(root_href=items[0].id,
                               catalog_type=CatalogType.RELATIVE_PUBLISHED)

    catalog.describe()
    
    catalogs.append(catalog.get_self_href())

2020-11-25T10:14:15 DEBUG    Starting new HTTPS connection (1): terradue-rtd.gitlab.io:443
2020-11-25T10:14:16 DEBUG    https://terradue-rtd.gitlab.io:443 "GET /sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414.json HTTP/1.1" 200 17169
2020-11-25T10:14:16 DEBUG    Starting new HTTPS connection (1): terradue-rtd.gitlab.io:443


* <Catalog id=S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414>
  * <Item id=S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414>


2020-11-25T10:14:16 DEBUG    https://terradue-rtd.gitlab.io:443 "GET /sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348/S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348.json HTTP/1.1" 200 17119
2020-11-25T10:14:16 DEBUG    Starting new HTTPS connection (1): terradue-rtd.gitlab.io:443


* <Catalog id=S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348>
  * <Item id=S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348>


2020-11-25T10:14:17 DEBUG    https://terradue-rtd.gitlab.io:443 "GET /sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720.json HTTP/1.1" 200 17116
2020-11-25T10:14:17 DEBUG    Starting new HTTPS connection (1): terradue-rtd.gitlab.io:443


* <Catalog id=S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720>
  * <Item id=S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720>


2020-11-25T10:14:17 DEBUG    https://terradue-rtd.gitlab.io:443 "GET /sentinel-s2-l2a-cogs/48/M/YT/S2A_MSIL2A_20200906T025551_N0214_R032_T48MYT_20200906T063429/S2A_MSIL2A_20200906T025551_N0214_R032_T48MYT_20200906T063429.json HTTP/1.1" 200 20153


* <Catalog id=S2A_MSIL2A_20200906T025551_N0214_R032_T48MYT_20200906T063429>
  * <Item id=S2A_MSIL2A_20200906T025551_N0214_R032_T48MYT_20200906T063429>


In [6]:
catalogs

['/workspace/charter-pe/scombi-do/demo/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414/catalog.json',
 '/workspace/charter-pe/scombi-do/demo/S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348/catalog.json',
 '/workspace/charter-pe/scombi-do/demo/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720/catalog.json',
 '/workspace/charter-pe/scombi-do/demo/S2A_MSIL2A_20200906T025551_N0214_R032_T48MYT_20200906T063429/catalog.json']

### Scombi-do

#### Simple RGB combination

Create a profile for a simple RGB composite

In [28]:
expressions = ['(interp v1 (asarray 0 10000) (asarray 0 1))', 
               '(interp v2 (asarray 0 10000) (asarray 0 1))',
               '(interp v3 (asarray 0 10000) (asarray 0 1))']

Define the bands for the RGB channels using the common band names

In [29]:
bands = ['red', 'green', 'blue']

Define the input local STAC catalogs

In [30]:
channel_inputs = ['/workspace/charter-pe/scombi-do/demo/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414/catalog.json',
                  '/workspace/charter-pe/scombi-do/demo/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414/catalog.json',
                  '/workspace/charter-pe/scombi-do/demo/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414/catalog.json']

In [31]:
color = 'Gamma RGB 3.5 Saturation 1.4 Sigmoidal RGB 15 0.35' 

In [40]:
aoi = 'POLYGON((136.707 -35.991,136.707 -35.804,137.071 -35.804,137.071 -35.991,136.707 -35.991))'

In [41]:
params = dict()

params['channel_inputs'] = channel_inputs
params['bands'] = bands
params['s_expressions'] = expressions
params['color'] = color
params['aoi'] = aoi


In [33]:
result = main(**params)

2020-11-25T10:32:19 INFO     <Item id=S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414>
2020-11-25T10:32:19 INFO     <Item id=S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414>
2020-11-25T10:32:19 INFO     <Item id=S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414>
2020-11-25T10:32:20 INFO     Rescaling and COG for input assets
2020-11-25T10:32:20 INFO     Getting band red from /vsicurl/https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/53/H/PA/2019/10/S2B_53HPA_20191012_0_L2A/B04.tif
2020-11-25T10:32:23 INFO     Getting band green from /vsicurl/https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/53/H/PA/2019/10/S2B_53HPA_20191012_0_L2A/B03.tif
2020-11-25T10:32:25 INFO     Getting band blue from /vsicurl/https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/53/H/PA/2019/10/S2B_53HPA_20191012_0_L2A/B02.tif
2020-11-25T10:32:27 INFO     Build VRT
2020-11-25T10:32:27 INFO     10.0
2020-11-25T10:32:27 INF

In [34]:
result

'/workspace/charter-pe/scombi-do/demo/catalog.json'

In [35]:
item = next(read_file(result).get_items())

In [42]:
dsw = gdal.Warp('/vsimem/warp.tif',
              item.get_assets()['rgb'].get_absolute_href(), 
              dstSRS='EPSG:4326',
              format='GTiff',
               dstAlpha=True)

In [43]:
ds = gdal.Open('/vsimem/warp.tif')

_bands = []
for band_index in [1,2,3,4]:

    band = ds.GetRasterBand(band_index)
    w = band.XSize
    h = band.YSize
    _bands.append(band.ReadAsArray().astype(np.uint8))

rgb_uint8 = np.dstack(_bands).astype(np.uint8)

im = Image.fromarray(rgb_uint8)

f = BytesIO()

im.save(f, 'png')
data = b64encode(f.getvalue())

ds = None
dsw = None
del(ds)
del(dsw)

In [44]:
m = Map(center=(shape(item.geometry).centroid.y, 
                shape(item.geometry).centroid.x), 
                zoom=10)

image = ImageOverlay(
            url=b'data:image/png;base64,' + data,
            bounds=((shape(item.geometry).bounds[1], 
                     shape(item.geometry).bounds[0]), 
                    (shape(item.geometry).bounds[3], 
                     shape(item.geometry).bounds[2]))
        )
m.add_layer(image)
m

Map(center=[-35.8975, 136.88900000000004], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in…

#### Normalized difference

In [45]:
expressions = ['(interp (/ (- v1 v2) (+ v1 v2)) (asarray -1 1) (asarray 0 1))']

Define the bands for the RGB channels using the common band names

In [46]:
bands = ['nir', 'red']

Define the input local STAC catalogs

In [53]:
channel_inputs = ['/workspace/charter-pe/scombi-do/demo/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414/catalog.json',
                  '/workspace/charter-pe/scombi-do/demo/S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414/catalog.json']

In [54]:
aoi = 'POLYGON((136.707 -35.991,136.707 -35.804,137.071 -35.804,137.071 -35.991,136.707 -35.991))'

In [55]:
params = dict()

params['channel_inputs'] = channel_inputs
params['bands'] = bands
params['s_expressions'] = expressions
params['aoi'] = aoi
params['lut'] = 'viridis'

In [56]:
result = main(**params)

2020-11-25T10:51:04 INFO     <Item id=S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414>
2020-11-25T10:51:04 INFO     <Item id=S2B_MSIL2A_20191012T004709_N0213_R102_T53HPA_20191012T023414>
2020-11-25T10:51:05 INFO     Rescaling and COG for input assets
2020-11-25T10:51:05 INFO     Getting band nir from /vsicurl/https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/53/H/PA/2019/10/S2B_53HPA_20191012_0_L2A/B08.tif
2020-11-25T10:51:08 INFO     Getting band red from /vsicurl/https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/53/H/PA/2019/10/S2B_53HPA_20191012_0_L2A/B04.tif
2020-11-25T10:51:11 INFO     Build VRT
2020-11-25T10:51:11 INFO     10.0
2020-11-25T10:51:11 INFO     Pimp me
2020-11-25T10:51:11 INFO     2
2020-11-25T10:51:12 INFO     Applying look-up table
2020-11-25T10:51:12 INFO     Adding band 1 of 3
2020-11-25T10:51:12 INFO     Adding band 2 of 3
2020-11-25T10:51:12 INFO     Adding band 3 of 3
2020-11-25T10:51:17 INFO     STAC
2020-11

In [60]:
item = next(read_file(result).get_items())

item

<Item id=combi>

In [61]:
dsw = gdal.Warp('/vsimem/warp.tif',
              item.get_assets()['rgb'].get_absolute_href(), 
              dstSRS='EPSG:4326',
              format='GTiff',
               dstAlpha=True)

ds = gdal.Open('/vsimem/warp.tif')

_bands = []
for band_index in [1,2,3,4]:

    band = ds.GetRasterBand(band_index)
    w = band.XSize
    h = band.YSize
    _bands.append(band.ReadAsArray().astype(np.uint8))

rgb_uint8 = np.dstack(_bands).astype(np.uint8)

im = Image.fromarray(rgb_uint8)

f = BytesIO()

im.save(f, 'png')
data = b64encode(f.getvalue())

ds = None
dsw = None
del(ds)
del(dsw)

In [62]:
m = Map(center=(shape(item.geometry).centroid.y, 
                shape(item.geometry).centroid.x), 
                zoom=10)

image = ImageOverlay(
            url=b'data:image/png;base64,' + data,
            bounds=((shape(item.geometry).bounds[1], 
                     shape(item.geometry).bounds[0]), 
                    (shape(item.geometry).bounds[3], 
                     shape(item.geometry).bounds[2]))
        )
m.add_layer(image)
m

Map(center=[-35.8975, 136.88900000000004], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in…